# **Data Cleaning**

## **Installing and importing dependencies and dataframe**



In [ ]:
!pip install dill &> /dev/null

In [ ]:
import pandas as pd
import dill

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/DC_Inter/'
with open(path+'football_stats_2022_2023.pkl','rb') as f:
    df = dill.load(f)

In [ ]:
df.head()

league season      team                player  \
                                                                           
0  ARG-Primera División Argentina   2223  Aldosivi           Andrés Ríos   
1  ARG-Primera División Argentina   2223  Aldosivi  Bautista Kociubinski   
2  ARG-Primera División Argentina   2223  Aldosivi        Brian Martínez   
3  ARG-Primera División Argentina   2223  Aldosivi          David Torres   
4  ARG-Primera División Argentina   2223  Aldosivi        Edwin Mosquera   

  nation    pos age  born Playing Time_x         ... Performance_y           \
                                      MP Starts  ...           Crs Int TklW   
0    ARG  FW,MF  32  1989              6      4  ...             4   1    2   
1    ARG     MF  20  2001              8      8  ...             5  12   13   
2    ARG  FW,MF  22  1999             18     14  ...            56   5    8   
3    ARG  FW,DF  20  2001              3      0  ...             0   0    0   
4    COL     FW  20  2001              1      0  ...             2   0    1   

                       Aerial Duels              
  PKwon PKcon OG Recov          Won  Lost  Won%  
0     0     0  0    16            9    10  47.4  
1     0     0  0    59           12     8  60.0  
2  <NA>  <NA>  0  <NA>         <NA>  <NA>  <NA>  
3     0     0  0     1            2     1  66.7  
4     0     0  0     1            0     0  <NA>  

[5 rows x 191 columns]

## **Dataset Manipulation**

We note that the retrieved dataframe has multi-index, so we are interested in removing it and having each type of statistic under a specific column name.

In [ ]:
df.columns = [''.join(col) for col in df.columns]

In [ ]:
df.columns

Index(['league', 'season', 'team', 'player', 'nation', 'pos', 'age', 'born',
       'Playing Time_xMP', 'Playing Time_xStarts',
       ...
       'Performance_yCrs', 'Performance_yInt', 'Performance_yTklW',
       'Performance_yPKwon', 'Performance_yPKcon', 'Performance_yOG',
       'Performance_yRecov', 'Aerial DuelsWon', 'Aerial DuelsLost',
       'Aerial DuelsWon%'],
      dtype='object', length=191)

In addition, the dataframe has duplicate columns due to the merging done earlier. We are interested in removing those duplicate columns.

In [ ]:
# Transpose the DataFrame to make columns as rows
transposed_df = df.transpose()

# Find duplicate columns (excluding the first occurrence)
duplicate_columns = transposed_df.duplicated(keep='first')

# Get the unique column names
unique_columns = transposed_df[~duplicate_columns].index

# Transpose the DataFrame back and keep only the unique columns
df = df[unique_columns].copy()

In [ ]:
df[df['player'] == 'Ike Ugbo']

,league,season,team,player,nation,pos,age,born,Playing Time_xMP,Playing Time_xStarts,...,Performance_yFls,Performance_yFld,Performance_yOff,Performance_yPKwon,Performance_yPKcon,Performance_yOG,Performance_yRecov,Aerial DuelsWon,Aerial DuelsLost,Aerial DuelsWon%
4073,FRA-Ligue 1,2223,Troyes,Ike Ugbo,CAN,"FW,MF",23,1998,25,9,...,13,10,4,0,0,0,15,9,41,18.0


We remove *_x* and *_y* at the end of the name's columns through the use of a regular expression.

In [ ]:
df.columns = df.columns.str.replace(r'_x', ' ', regex = True)
df.columns = df.columns.str.replace(r'_y',' ', regex = True)

In [ ]:
df.head()

,league,season,team,player,nation,pos,age,born,Playing Time MP,Playing Time Starts,...,Performance Fls,Performance Fld,Performance Off,Performance PKwon,Performance PKcon,Performance OG,Performance Recov,Aerial DuelsWon,Aerial DuelsLost,Aerial DuelsWon%
0,ARG-Primera División Argentina,2223,Aldosivi,Andrés Ríos,ARG,"FW,MF",32,1989,6,4,...,12,8,2,0,0,0,16,9,10,47.4
1,ARG-Primera División Argentina,2223,Aldosivi,Bautista Kociubinski,ARG,MF,20,2001,8,8,...,11,5,0,0,0,0,59,12,8,60.0
2,ARG-Primera División Argentina,2223,Aldosivi,Brian Martínez,ARG,"FW,MF",22,1999,18,14,...,9,21,3,<NA>,<NA>,0,<NA>,<NA>,<NA>,<NA>
3,ARG-Primera División Argentina,2223,Aldosivi,David Torres,ARG,"FW,DF",20,2001,3,0,...,0,0,1,0,0,0,1,2,1,66.7
4,ARG-Primera División Argentina,2223,Aldosivi,Edwin Mosquera,COL,FW,20,2001,1,0,...,1,1,0,0,0,0,1,0,0,<NA>


We are also interested in giving more understandable and clear names to the columns; although this is a time-consuming process, it will result in a better understanding of the dataframe and the statistics within it.

In [ ]:
columns = {
    'league' : 'League', 'season' : 'Season', 'team' : 'Team', 'player' : 'Player','nation' : 'Nation', 'pos' : 'Position', 'age' : 'Age', 'born' : 'Born',
    'Playing Time MP' : 'Match Played','Playing Time Starts' : 'Match Started', 'Playing Time Min' : 'Minutes Played', 'Playing Time 90s' : 'Minutes Played/90',
    'Performance Gls' : 'Goals', 'Performance Ast' : 'Assists', 'Performance G+A' : 'Goals + Assists', 'Performance G-PK' : 'Goals (No penalties)', 'Performance PK' : 'Penalties',
    'Performance PKatt' : 'Penalties Attempted', 'Performance CrdY' : 'Yellow cards', 'Performance CrdR' : 'Red cards', 'Expected xG' : 'Expected goals',
    'Expected npxG' : 'Expected goals (No penalties)', 'Expected xAG' : 'Expected assisted goals', 'ProgressionPrgC' : 'Progressive carries', 'ProgressionPrgP':'Progressive passes',
    'ProgressionPrgR' : 'Progressive passes received', 'Per 90 MinutesGls' : 'Goals/90', 'Per 90 MinutesAst' : 'Assists/90', 'Per 90 MinutesG+A' : 'Goals+Assists/90',
    'Per 90 MinutesG-PK' : 'Goals(no penalties)/90', 'Per 90 MinutesG+A-PK' : 'Goals+Assists(no penalties)/90', 'Per 90 MinutesxG' : 'Expected goals/90', 'Per 90 MinutesxAG' : 'Expected assisted goals/90',
    'Per 90 MinutesxG+xAG' : 'Expected goals + Expected assisted goals/90', 'Per 90 MinutesnpxG' : 'Expected goals (no penalties)/90', 'Per 90 MinutesnpxG+xAG' : 'Expected goals + Expected assisted goals (no penalties)/90',
    'StandardSh' : 'Shots', 'StandardSoT' : 'Shots on target', 'StandardSoT%' : 'Shots on target %', 'StandardSh/90' : 'Shots/90', 'StandardSoT/90' : 'Shots on target/90',
    'StandardG/Sh' : 'Goals per shot', 'StandardG/SoT' : 'Goals per shots on target', 'StandardDist' : 'Average shot distance', 'StandardFK' : 'Shots from free kicks',
    'Expected npxG/Sh': 'Expected goals (no penalties) per shots', 'Expected G-xG': 'Goals minus expected goals', 'Expected np:G-xG': 'No penalties goals minus no penalties exp goals',
    'TotalCmp': 'Passes completed', 'TotalAtt': 'Passes attempted', 'TotalCmp%': 'Passes completed%', 'TotalTotDist': 'Total passing distance', 'TotalPrgDist': 'Progressive passing distance',
    'ShortCmp': 'Short passes completed', 'ShortAtt': 'Short passes attempted', 'ShortCmp%':'Short passes completed%', 'MediumCmp': 'Medium passes completed', 'MediumAtt': 'Medium passes attempted',
    'MediumCmp%':'Medium passes completed%', 'LongCmp': 'Long passes completed', 'LongAtt': 'Long passes attempted', 'LongCmp%':'Long passes completed%', 'ExpectedxA' : 'Expected assists',
    'ExpectedA-xAG' : 'Expected assists minus expected assisted goals', 'KP': 'Key passes', '1/3': 'Passes into final third', 'PPA' : 'Passes into penalty area',
    'CrsPA': 'Crosses into penalty area', 'Pass TypesLive' : 'Live ball passes', 'Pass TypesDead' : 'Dead ball passes', 'Pass TypesFK' : 'Passes from free kicks',
    'Pass TypesTB' : 'Through balls', 'Pass TypesSw' : 'Switches', 'Pass TypesCrs' : 'Crosses', 'Pass TypesTI' : 'Throw-ins taken', 'Pass TypesCK' : 'Corner kicks',
    'Corner KicksIn' : 'In corner kicks', 'Corner KicksOut' : 'Out corner kicks', 'Corner KicksStr' : 'Straight corner kicks', 'OutcomesOff' : 'Passes offside',
    'OutcomesBlocks' : 'Passes blocked', 'SCASCA' : 'Shot creating actions', 'SCASCA90' : 'Shot creating actions/90', 'SCA TypesPassLive' : '(SCA) Live ball passes',
    'SCA TypesPassDead' : '(SCA) Dead ball passes', 'SCA TypesTO' : '(SCA) Take ons', 'SCA TypesSh' : '(SCA) Shot', 'SCA TypesFld' : '(SCA) Fouls drawn',
    'SCA TypesDef' : '(SCA) Defensive action', 'GCAGCA' : 'Goal creating actions', 'GCAGCA90' : 'Goal creating actions/90', 'GCA TypesPassLive' : '(GCA) Live ball passes',
    'GCA TypesPassDead' : '(GCA) Dead ball passes', 'GCA TypesTO' : '(GCA) Take ons', 'GCA TypesSh' : '(GCA) Shot', 'GCA TypesFld' : '(GCA) Fouls drawn',
    'GCA TypesDef' : '(GCA) Defensive action', 'TacklesTkl' : 'Tackles', 'TacklesTklW' : 'Tackles won', 'TacklesDef 3rd' : 'Tackles in defensive third',
    'TacklesMid 3rd' : 'Tackles in middle third', 'TacklesAtt 3rd' : 'Tackles in attacking third', 'ChallengesTkl' : 'Dribblers tackled', 'ChallengesAtt' : 'Dribblers challenged',
    'ChallengesTkl%' : '% of dribblers tackled', 'ChallengesLost' : 'Challenges lost', 'BlocksBlocks' : 'Blocks', 'BlocksSh' : 'Shot blocked', 'BlocksSh' : 'Shots blocked',
    'BlocksPass' : 'Passes blocked' , 'Int' : 'Interceptions', 'Tkl+Int' : 'Tackles and interceptions', 'Clr' : 'Clearances', 'Err' : 'Errors', 'TouchesTouches' : 'Touches',
    'TouchesDef Pen' : 'Touches in defensive penalty area', 'TouchesDef 3rd' : 'Touches in defensive third', 'TouchesMid 3rd' : 'Touches in middle third',
    'TouchesAtt 3rd' : 'Touches in attacking third', 'TouchesAtt Pen' : 'Touches in attacking penalty area', 'TouchesLive' : 'Live ball touches', 'Take-OnsAtt' : 'Take ons attempted',
    'Take-OnsSucc' : 'Take ons successful', 'Take-OnsSucc%' : '% of successful take ons', 'Take-OnsTkld' : 'Tackles during take ons', 'Take-OnsTkld%' : '% of tackles during take ons',
    'CarriesCarries' : 'Carries', 'CarriesTotDist' : 'Total carrying distance', 'CarriesPrgDist' : 'Progressive carrying distance', 'Carries1/3' : 'Carries into final third',
    'CarriesCPA' : 'Carries into penalty area', 'CarriesMis' : 'Miscontrols', 'CarriesDis' : 'Dispossessed', 'ReceivingRec' : 'Passes received' , 'Playing Time Mn/MP' : 'Minutes per match played',
    'Playing Time Min%' : 'Percentage of minutes played', 'StartsMn/Start' : 'Minutes per match started', 'StartsCompl' : 'Complete matches played', 'SubsSubs' : 'Games as sub',
    'SubsMn/Sub' : 'Minutes per substitution', 'SubsunSub' : 'Matches as unused sub', 'Team SuccessPPM' : 'Points per match', 'Team SuccessonG' : 'Goals scored while on pitch',
    'Team SuccessonGA':'Goals allowed while on pitch', 'Team Success+/-' : 'Goals scored minus goals allowed while on pitch', 'Team Success+/-90' : 'Goals scored minus goals allowed while on pitch/90',
    'Team SuccessOn-Off' : 'Plus/minus net per 90 minutes', 'Team Success (xG)onxG' : 'xG (on pitch)', 'Team Success (xG)onxGA' : 'xGA (on pitch)',
    'Team Success (xG)xG+/-' : 'xG minus xGA (on pitch)', 'Team Success (xG)xG+/-90' : 'xG minus xGA / 90', 'Team Success (xG)On-Off' : 'xG plus/minus net per 90 minutes',
    'Performance 2CrdY' : 'Second yellow cards', 'Performance Fls' : 'Fouls committed', 'Performance Fld' : 'Fouls drawn', 'Performance Off' : 'Offsides', 'Performance PKwon' : 'Penalty kicks won',
    'Performance PKcon' : 'Penalty kicks conceded', 'Performance OG' : 'Own goals', 'Performance Recov' : 'Ball recoveries', 'Aerial DuelsWon' : 'Aerial duels won',
    'Aerial DuelsLost' : 'Aerial duels lost', 'Aerial DuelsWon%' : '% of aerial duels won'
}

In [ ]:
df.rename(columns=columns, inplace=True)

In [ ]:
df.head()

,League,Season,Team,Player,Nation,Position,Age,Born,Match Played,Match Started,...,Fouls committed,Fouls drawn,Offsides,Penalty kicks won,Penalty kicks conceded,Own goals,Ball recoveries,Aerial duels won,Aerial duels lost,% of aerial duels won
0,ARG-Primera División Argentina,2223,Aldosivi,Andrés Ríos,ARG,"FW,MF",32,1989,6,4,...,12,8,2,0,0,0,16,9,10,47.4
1,ARG-Primera División Argentina,2223,Aldosivi,Bautista Kociubinski,ARG,MF,20,2001,8,8,...,11,5,0,0,0,0,59,12,8,60.0
2,ARG-Primera División Argentina,2223,Aldosivi,Brian Martínez,ARG,"FW,MF",22,1999,18,14,...,9,21,3,<NA>,<NA>,0,<NA>,<NA>,<NA>,<NA>
3,ARG-Primera División Argentina,2223,Aldosivi,David Torres,ARG,"FW,DF",20,2001,3,0,...,0,0,1,0,0,0,1,2,1,66.7
4,ARG-Primera División Argentina,2223,Aldosivi,Edwin Mosquera,COL,FW,20,2001,1,0,...,1,1,0,0,0,0,1,0,0,<NA>


## **Data Filtering**

We will now filter the dataframe according to the retrieved statistics. We are going to discard statistics that we are not interested in and that are not really related to defenders (e.g., wanting to find the players most similar to Acerbi, we are not interested in the number of xGs of a player, but rather in how many times he tries a long pass to the striker rather than a short pass to the fullback or the midfielder).

The following are the statistics we will consider in our analysis:

In [ ]:
df = df[['League','Season', 'Team','Player','Nation','Position','Age','Born','Match Played','Match Started','Minutes Played','Yellow cards', 'Red cards', 'Progressive carries',
'Progressive passes', 'Passes completed', 'Passes attempted', 'Passes completed%','Total passing distance','Progressive passing distance',
'Short passes completed', 'Short passes attempted','Short passes completed%','Medium passes completed','Medium passes attempted','Medium passes completed%',
'Long passes completed','Long passes attempted','Long passes completed%', 'Key passes','Passes into final third','Passes into penalty area','Passes offside','Passes blocked',
'Tackles','Tackles won','Tackles in defensive third','Tackles in middle third','Tackles in attacking third','Dribblers tackled','Dribblers challenged',
'% of dribblers tackled','Challenges lost','Blocks','Shots blocked','Interceptions','Tackles and interceptions','Clearances','Errors','Touches','Touches in defensive penalty area',
'Touches in defensive third','Touches in middle third','Touches in attacking third','Carries','Total carrying distance','Progressive carrying distance','Carries into final third','Carries into penalty area',
'Miscontrols','Dispossessed','Second yellow cards','Fouls committed','Penalty kicks conceded','Own goals','Ball recoveries','Aerial duels won','Aerial duels lost','% of aerial duels won']]

We are also going to filter the dataframe by going to consider defenders (or those that FBref considers as such). On some occasions there may be more than one role within the *Position* column, so we are going to use a regular expression to go and identify the players that FBref considers to be also defenders.

We will also only consider players who have played at least 500 minutes in their respective leagues (a little more than 5 games), to try not to have statistics distorted by the small amount of minutes.

In [ ]:
df1 = df[df['Position'].str.contains('DF')].reset_index(drop = True)

In [ ]:
df2 = df1[df1['Minutes Played'] >= 500].reset_index(drop=True)
df2

,League,Season,Team,Player,Nation,Position,Age,Born,Match Played,Match Started,...,Miscontrols,Dispossessed,Second yellow cards,Fouls committed,Penalty kicks conceded,Own goals,Ball recoveries,Aerial duels won,Aerial duels lost,% of aerial duels won
0,ARG-Primera División Argentina,2223,Aldosivi,Fernando Román,PAR,DF,23,1998,8,7,...,12,9,0,8,0,0,49,10,11,47.6
1,ARG-Primera División Argentina,2223,Aldosivi,Ian Escobar,ARG,DF,25,1996,12,11,...,23,4,1,8,1,0,49,9,9,50.0
2,ARG-Primera División Argentina,2223,Aldosivi,Joaquín Indacoechea,ARG,DF,21,2000,10,6,...,8,6,0,8,0,0,45,9,8,52.9
3,ARG-Primera División Argentina,2223,Aldosivi,Mario López Quintana,PAR,DF,26,1995,12,11,...,7,1,0,19,0,1,61,30,15,66.7
4,ARG-Primera División Argentina,2223,Aldosivi,Nicolás Valentini,ARG,DF,20,2001,25,25,...,17,2,1,21,2,0,117,95,43,68.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2553,USA-Major League Soccer,2223,Vancouver,Julian Gressel,USA,"MF,DF",28,1993,13,11,...,13,7,0,7,0,0,52,6,7,46.2
2554,USA-Major League Soccer,2223,Vancouver,Marcus Godinho,CAN,DF,24,1997,23,12,...,17,12,0,10,0,0,53,8,11,42.1
2555,USA-Major League Soccer,2223,Vancouver,Ranko Veselinović,SRB,DF,22,1999,31,31,...,9,2,0,22,3,0,112,49,45,52.1
2556,USA-Major League Soccer,2223,Vancouver,Ryan Raposo,CAN,DF,22,1999,30,17,...,33,9,0,6,0,0,86,8,18,30.8


Let us now look at the number of players with missing values.
We can see that the number of players with missing values is 12, so let us eliminate the aforementioned, as they are a very small part of the dataframe (just over 0.4%).

In [ ]:
df2.isna().sum().max()

12

In [ ]:
df3 = df2[~df2.isna().any(axis=1)]
df3.head()

,League,Season,Team,Player,Nation,Position,Age,Born,Match Played,Match Started,...,Miscontrols,Dispossessed,Second yellow cards,Fouls committed,Penalty kicks conceded,Own goals,Ball recoveries,Aerial duels won,Aerial duels lost,% of aerial duels won
0,ARG-Primera División Argentina,2223,Aldosivi,Fernando Román,PAR,DF,23,1998,8,7,...,12,9,0,8,0,0,49,10,11,47.6
1,ARG-Primera División Argentina,2223,Aldosivi,Ian Escobar,ARG,DF,25,1996,12,11,...,23,4,1,8,1,0,49,9,9,50.0
2,ARG-Primera División Argentina,2223,Aldosivi,Joaquín Indacoechea,ARG,DF,21,2000,10,6,...,8,6,0,8,0,0,45,9,8,52.9
3,ARG-Primera División Argentina,2223,Aldosivi,Mario López Quintana,PAR,DF,26,1995,12,11,...,7,1,0,19,0,1,61,30,15,66.7
4,ARG-Primera División Argentina,2223,Aldosivi,Nicolás Valentini,ARG,DF,20,2001,25,25,...,17,2,1,21,2,0,117,95,43,68.8


Next, we divide the dataframe into two, one with statistics that might be useful for analysis and another with the players' personal data (such as team, position, name, and date of birth).

In each case, we leave an index in both dataframes in case later we are interested in merging them again.

In [ ]:
columns_id = ['League','Season', 'Team','Player','Nation','Position','Age','Born']
columns_stat =['Match Played', 'Match Started','Minutes Played','Yellow cards', 'Red cards', 'Progressive carries',
'Progressive passes', 'Passes completed', 'Passes attempted', 'Passes completed%','Total passing distance','Progressive passing distance',
'Short passes completed', 'Short passes attempted','Short passes completed%','Medium passes completed','Medium passes attempted','Medium passes completed%',
'Long passes completed','Long passes attempted','Long passes completed%', 'Key passes','Passes into final third','Passes into penalty area','Passes offside','Passes blocked',
'Tackles','Tackles won','Tackles in defensive third','Tackles in middle third','Tackles in attacking third','Dribblers tackled','Dribblers challenged',
'% of dribblers tackled','Challenges lost','Blocks','Shots blocked','Interceptions','Tackles and interceptions','Clearances','Errors','Touches','Touches in defensive penalty area',
'Touches in defensive third','Touches in middle third','Touches in attacking third','Carries','Total carrying distance','Progressive carrying distance','Carries into final third','Carries into penalty area',
'Miscontrols','Dispossessed','Second yellow cards','Fouls committed','Penalty kicks conceded','Own goals','Ball recoveries','Aerial duels won','Aerial duels lost','% of aerial duels won']

In [ ]:
players = df3[columns_id]
players.reset_index(inplace=True)

In [ ]:
players

,index,League,Season,Team,Player,Nation,Position,Age,Born
0,0,ARG-Primera División Argentina,2223,Aldosivi,Fernando Román,PAR,DF,23,1998
1,1,ARG-Primera División Argentina,2223,Aldosivi,Ian Escobar,ARG,DF,25,1996
2,2,ARG-Primera División Argentina,2223,Aldosivi,Joaquín Indacoechea,ARG,DF,21,2000
3,3,ARG-Primera División Argentina,2223,Aldosivi,Mario López Quintana,PAR,DF,26,1995
4,4,ARG-Primera División Argentina,2223,Aldosivi,Nicolás Valentini,ARG,DF,20,2001
...,...,...,...,...,...,...,...,...,...
2541,2553,USA-Major League Soccer,2223,Vancouver,Julian Gressel,USA,"MF,DF",28,1993
2542,2554,USA-Major League Soccer,2223,Vancouver,Marcus Godinho,CAN,DF,24,1997
2543,2555,USA-Major League Soccer,2223,Vancouver,Ranko Veselinović,SRB,DF,22,1999
2544,2556,USA-Major League Soccer,2223,Vancouver,Ryan Raposo,CAN,DF,22,1999


In [ ]:
with open(path+'players.pkl', 'wb') as f: #Saving the dataframe with personal data of the players
    dill.dump(players, f)

In [ ]:
players_stats = df3[columns_stat]
players_stats.reset_index(inplace = True)

In [ ]:
players_stats

,index,Match Played,Match Started,Minutes Played,Yellow cards,Red cards,Progressive carries,Progressive passes,Passes completed,Passes attempted,...,Miscontrols,Dispossessed,Second yellow cards,Fouls committed,Penalty kicks conceded,Own goals,Ball recoveries,Aerial duels won,Aerial duels lost,% of aerial duels won
0,0,8,7,638,2,0,7,25,213,322,...,12,9,0,8,0,0,49,10,11,47.6
1,1,12,11,965,4,1,27,34,295,468,...,23,4,1,8,1,0,49,9,9,50.0
2,2,10,6,528,2,0,5,18,196,277,...,8,6,0,8,0,0,45,9,8,52.9
3,3,12,11,1035,3,0,3,29,334,471,...,7,1,0,19,0,1,61,30,15,66.7
4,4,25,25,2185,6,1,15,96,690,1088,...,17,2,1,21,2,0,117,95,43,68.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2541,2553,13,11,972,1,0,26,51,352,534,...,13,7,0,7,0,0,52,6,7,46.2
2542,2554,23,12,1019,1,0,36,56,364,456,...,17,12,0,10,0,0,53,8,11,42.1
2543,2555,31,31,2761,5,0,4,49,920,1083,...,9,2,0,22,3,0,112,49,45,52.1
2544,2556,30,17,1440,2,0,43,62,549,741,...,33,9,0,6,0,0,86,8,18,30.8


In [ ]:
players_stats = players_stats.loc[:,~players_stats.columns.duplicated()].copy() # Checking if there are any columns with the same name

In [ ]:
players_stats.head()

,index,Match Played,Match Started,Minutes Played,Yellow cards,Red cards,Progressive carries,Progressive passes,Passes completed,Passes attempted,...,Miscontrols,Dispossessed,Second yellow cards,Fouls committed,Penalty kicks conceded,Own goals,Ball recoveries,Aerial duels won,Aerial duels lost,% of aerial duels won
0,0,8,7,638,2,0,7,25,213,322,...,12,9,0,8,0,0,49,10,11,47.6
1,1,12,11,965,4,1,27,34,295,468,...,23,4,1,8,1,0,49,9,9,50.0
2,2,10,6,528,2,0,5,18,196,277,...,8,6,0,8,0,0,45,9,8,52.9
3,3,12,11,1035,3,0,3,29,334,471,...,7,1,0,19,0,1,61,30,15,66.7
4,4,25,25,2185,6,1,15,96,690,1088,...,17,2,1,21,2,0,117,95,43,68.8


## **Feature Creation (per90 stats)**

Most of the statistics retrieved are raw statistics (all with the exception of percentages), that is, they are relative to the entire season; these statistics can be misleading, for example, it is obvious that a player who has played 1000 minutes in the season will have fewer passes made than one who has played 3000 minutes.

We then go on to transform the raw statistics into *per90* statistics, which tell us the production of a particular stat over a 90 minute period. These type of stats are able to give better clues about the player's performance and are more useful when comparing players.

The computation is done in the following way:
\begin{equation}
stats_{per90} = \frac{stats_{raw}}{minutes played} \times 90
\end{equation}

This computation is done on all the statistics with the exception of the percentages.

In [ ]:
list(players_stats.columns)

['index',
 'Match Played',
 'Match Started',
 'Minutes Played',
 'Yellow cards',
 'Red cards',
 'Progressive carries',
 'Progressive passes',
 'Passes completed',
 'Passes attempted',
 'Passes completed%',
 'Total passing distance',
 'Progressive passing distance',
 'Short passes completed',
 'Short passes attempted',
 'Short passes completed%',
 'Medium passes completed',
 'Medium passes attempted',
 'Medium passes completed%',
 'Long passes completed',
 'Long passes attempted',
 'Long passes completed%',
 'Key passes',
 'Passes into final third',
 'Passes into penalty area',
 'Passes offside',
 'Passes blocked',
 'Tackles',
 'Tackles won',
 'Tackles in defensive third',
 'Tackles in middle third',
 'Tackles in attacking third',
 'Dribblers tackled',
 'Dribblers challenged',
 '% of dribblers tackled',
 'Challenges lost',
 'Blocks',
 'Shots blocked',
 'Interceptions',
 'Tackles and interceptions',
 'Clearances',
 'Errors',
 'Touches',
 'Touches in defensive penalty area',
 'Touches in 

In [ ]:
players_stats['yellow_cards_per90'] = (players_stats['Yellow cards'] / players_stats['Minutes Played']) * 90
players_stats['red_cards_per90'] = (players_stats['Red cards'] / players_stats['Minutes Played']) * 90
players_stats['progressive_carries_per90'] = (players_stats['Progressive carries'] / players_stats['Minutes Played']) * 90
players_stats['progressive_passes_per90'] = (players_stats['Progressive passes'] / players_stats['Minutes Played']) * 90
players_stats['passes_attempted_per90'] = (players_stats['Passes attempted'] / players_stats['Minutes Played']) * 90
players_stats['short_passes_attempted_per90'] = (players_stats['Short passes attempted'] / players_stats['Minutes Played']) * 90
players_stats['medium_passes_attempted_per90'] = (players_stats['Medium passes attempted'] / players_stats['Minutes Played']) * 90
players_stats['long_passes_attempted_per90'] = (players_stats['Long passes attempted'] / players_stats['Minutes Played']) * 90
players_stats['total_passing_distance_per90'] = (players_stats['Total passing distance'] / players_stats['Minutes Played']) * 90
players_stats['progressive_passing_distance_per90'] = (players_stats['Progressive passing distance'] / players_stats['Minutes Played']) * 90
players_stats['key_passes_per90'] = (players_stats['Key passes'] / players_stats['Minutes Played']) * 90
players_stats['passes_into_final_third_per90'] = (players_stats['Passes into final third'] / players_stats['Minutes Played']) * 90
players_stats['passes_into_penalty_area_per90'] = (players_stats['Passes into penalty area'] / players_stats['Minutes Played']) * 90
players_stats['passes_offside_per90'] = (players_stats['Passes offside'] / players_stats['Minutes Played']) * 90
players_stats['passes_blocked_per90'] = (players_stats['Passes blocked'] / players_stats['Minutes Played']) * 90
players_stats['tackles_per90'] = (players_stats['Tackles'] / players_stats['Minutes Played']) * 90
players_stats['tackles_won_per90'] = (players_stats['Tackles won'] / players_stats['Minutes Played']) * 90
players_stats['tackles_defensive_third_per90'] = (players_stats['Tackles in defensive third'] / players_stats['Minutes Played']) * 90
players_stats['tackles_middle_third_per90'] = (players_stats['Tackles in middle third'] / players_stats['Minutes Played']) * 90
players_stats['tackles_offensive_third_per90'] = (players_stats['Tackles in attacking third'] / players_stats['Minutes Played']) * 90
players_stats['dribblers_challenged_per90'] = (players_stats['Dribblers challenged'] / players_stats['Minutes Played']) * 90
players_stats['challenges_lost_per90'] = (players_stats['Challenges lost'] / players_stats['Minutes Played']) * 90
players_stats['blocks_per90'] = (players_stats['Blocks'] / players_stats['Minutes Played']) * 90
players_stats['shots_blocked_per90'] = (players_stats['Shots blocked'] / players_stats['Minutes Played']) * 90
players_stats['interceptions_per90'] = (players_stats['Interceptions'] / players_stats['Minutes Played']) * 90
players_stats['tackles_interceptions_per90'] = (players_stats['Tackles and interceptions'] / players_stats['Minutes Played']) * 90
players_stats['clearances_per90'] = (players_stats['Clearances'] / players_stats['Minutes Played']) * 90
players_stats['errors_per90'] = (players_stats['Errors'] / players_stats['Minutes Played']) * 90
players_stats['touches_per90'] = (players_stats['Touches'] / players_stats['Minutes Played']) * 90
players_stats['touches_defensive_penalty_area_per90'] = (players_stats['Touches in defensive penalty area'] / players_stats['Minutes Played']) * 90
players_stats['touches_defensive_third_per90'] = (players_stats['Touches in defensive third'] / players_stats['Minutes Played']) * 90
players_stats['touches_middle_third_per90'] = (players_stats['Touches in middle third'] / players_stats['Minutes Played']) * 90
players_stats['touches_offensive_third_per90'] = (players_stats['Touches in attacking third'] / players_stats['Minutes Played']) * 90
players_stats['carries_per90'] = (players_stats['Carries'] / players_stats['Minutes Played']) * 90
players_stats['total_carrying_distance_per90'] = (players_stats['Total carrying distance'] / players_stats['Minutes Played']) * 90
players_stats['progressive_carrying_distance_per90'] = (players_stats['Progressive carrying distance'] / players_stats['Minutes Played']) * 90
players_stats['carries_final_third_per90'] = (players_stats['Carries into final third'] / players_stats['Minutes Played']) * 90
players_stats['carries_into_penalty_area_per90'] = (players_stats['Carries into penalty area'] / players_stats['Minutes Played']) * 90
players_stats['miscontrols_per90'] = (players_stats['Miscontrols'] / players_stats['Minutes Played']) * 90
players_stats['dispossessed_per90'] = (players_stats['Dispossessed'] / players_stats['Minutes Played']) * 90
players_stats['fouls_committed_per90'] = (players_stats['Fouls committed'] / players_stats['Minutes Played']) * 90
players_stats['penalties_conceded_per90'] = (players_stats['Penalty kicks conceded'] / players_stats['Minutes Played']) * 90
players_stats['own_goals_per90'] = (players_stats['Own goals'] / players_stats['Minutes Played']) * 90
players_stats['ball_recoveries_per90'] = (players_stats['Ball recoveries'] / players_stats['Minutes Played']) * 90
players_stats['Aerial duels'] = players_stats['Aerial duels won'] + players_stats['Aerial duels lost']
players_stats['aerial_duels_per90'] = (players_stats['Aerial duels'] / players_stats['Minutes Played']) * 90

In [ ]:
list(players_stats.columns)

['index',
 'Match Played',
 'Match Started',
 'Minutes Played',
 'Yellow cards',
 'Red cards',
 'Progressive carries',
 'Progressive passes',
 'Passes completed',
 'Passes attempted',
 'Passes completed%',
 'Total passing distance',
 'Progressive passing distance',
 'Short passes completed',
 'Short passes attempted',
 'Short passes completed%',
 'Medium passes completed',
 'Medium passes attempted',
 'Medium passes completed%',
 'Long passes completed',
 'Long passes attempted',
 'Long passes completed%',
 'Key passes',
 'Passes into final third',
 'Passes into penalty area',
 'Passes offside',
 'Passes blocked',
 'Tackles',
 'Tackles won',
 'Tackles in defensive third',
 'Tackles in middle third',
 'Tackles in attacking third',
 'Dribblers tackled',
 'Dribblers challenged',
 '% of dribblers tackled',
 'Challenges lost',
 'Blocks',
 'Shots blocked',
 'Interceptions',
 'Tackles and interceptions',
 'Clearances',
 'Errors',
 'Touches',
 'Touches in defensive penalty area',
 'Touches in 

We then create a new dataframe with all the statistics for 90 minutes and percentages.

In [ ]:
players_stats_per_90 = players_stats[['index',
 'yellow_cards_per90',
 'red_cards_per90',
 'progressive_carries_per90',
 'progressive_passes_per90',
 'passes_attempted_per90',
 'short_passes_attempted_per90',
 'Short passes completed%',
 'medium_passes_attempted_per90',
 'Medium passes completed%',
 'long_passes_attempted_per90',
 'Long passes completed%',
 'total_passing_distance_per90',
 'progressive_passing_distance_per90',
 'key_passes_per90',
 'passes_into_final_third_per90',
 'passes_into_penalty_area_per90',
 'passes_offside_per90',
 'passes_blocked_per90',
 'tackles_per90',
 'tackles_won_per90',
 'tackles_defensive_third_per90',
 'tackles_middle_third_per90',
 'tackles_offensive_third_per90',
 'dribblers_challenged_per90',
 '% of dribblers tackled',
 'challenges_lost_per90',
 'blocks_per90',
 'shots_blocked_per90',
 'interceptions_per90',
 'tackles_interceptions_per90',
 'clearances_per90',
 'errors_per90',
 'touches_per90',
 'touches_defensive_penalty_area_per90',
 'touches_defensive_third_per90',
 'touches_middle_third_per90',
 'touches_offensive_third_per90',
 'carries_per90',
 'total_carrying_distance_per90',
 'progressive_carrying_distance_per90',
 'carries_final_third_per90',
 'carries_into_penalty_area_per90',
 'miscontrols_per90',
 'dispossessed_per90',
 'fouls_committed_per90',
 'penalties_conceded_per90',
 'own_goals_per90',
 'ball_recoveries_per90',
 'aerial_duels_per90',
 '% of aerial duels won']]

In [ ]:
players_stats_per_90

,index,yellow_cards_per90,red_cards_per90,progressive_carries_per90,progressive_passes_per90,passes_attempted_per90,short_passes_attempted_per90,Short passes completed%,medium_passes_attempted_per90,Medium passes completed%,...,carries_final_third_per90,carries_into_penalty_area_per90,miscontrols_per90,dispossessed_per90,fouls_committed_per90,penalties_conceded_per90,own_goals_per90,ball_recoveries_per90,aerial_duels_per90,% of aerial duels won
0,0,0.282132,0.0,0.987461,3.526646,45.423197,15.940439,78.8,18.338558,73.1,...,0.705329,0.141066,1.69279,1.269592,1.128527,0.0,0.0,6.912226,2.962382,47.6
1,1,0.373057,0.093264,2.518135,3.170984,43.647668,16.041451,79.7,17.160622,63.6,...,1.678756,0.373057,2.145078,0.373057,0.746114,0.093264,0.0,4.569948,1.678756,50.0
2,2,0.340909,0.0,0.852273,3.068182,47.215909,20.965909,78.0,18.238636,76.6,...,0.511364,0.0,1.363636,1.022727,1.363636,0.0,0.0,7.670455,2.897727,52.9
3,3,0.26087,0.0,0.26087,2.521739,40.956522,9.913043,79.8,19.130435,82.3,...,0.086957,0.0,0.608696,0.086957,1.652174,0.0,0.086957,5.304348,3.913043,66.7
4,4,0.24714,0.04119,0.617849,3.954233,44.814645,9.144165,69.8,21.295195,74.1,...,0.370709,0.0,0.700229,0.08238,0.864989,0.08238,0.0,4.819222,5.684211,68.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2541,2553,0.092593,0.0,2.407407,4.722222,49.444444,17.222222,83.9,18.240741,71.1,...,1.203704,0.185185,1.203704,0.648148,0.648148,0.0,0.0,4.814815,1.203704,46.2
2542,2554,0.088322,0.0,3.179588,4.946026,40.274779,17.752699,89.1,17.487733,82.3,...,2.296369,0.441609,1.501472,1.059863,0.883219,0.0,0.0,4.68106,1.678116,42.1
2543,2555,0.162984,0.0,0.130388,1.597247,35.302427,6.617168,89.2,21.807316,92.5,...,0.097791,0.032597,0.293372,0.065194,0.717131,0.097791,0.0,3.650851,3.064107,52.1
2544,2556,0.125,0.0,2.6875,3.875,46.3125,20.0625,87.9,15.875,77.2,...,1.75,0.6875,2.0625,0.5625,0.375,0.0,0.0,5.375,1.625,30.8


In [ ]:
with open(path+'stats_per_90_clean.pkl', 'wb') as f: #Saving the dataframe with per90 stats and percentages
   dill.dump(players_stats_per_90, f)